## รับข้อมูลการผลิตไฟจากInverter

In [50]:
import numpy as np
import pandas as pd

uri = input("Enter data name to load (without extension): ")

if not uri.endswith('.xlsx'):
    uri += '.xlsx'

df = pd.read_excel(uri)
# โหลดไฟล์ใหม่จากผู้ใช้อัปโหลดล่าสุด

df.columns.name = None

# ค้นหาแถวที่มีชื่อคอลัมน์จริง (แถวที่มี "Start Time")
header_row_index = df[df.eq("Start Time").any(axis=1)].index[0]

# ตั้งค่าหัวตารางใหม่
df.columns = df.iloc[header_row_index]
df = df.iloc[header_row_index + 1:].reset_index(drop=True)

# เปลี่ยนชื่อคอลัมน์ให้เป็นมาตรฐาน
df.rename(columns={"Start Time": "Time"}, inplace=True)

# ตรวจสอบค่าดิบของคอลัมน์ Time ก่อนแปลง
df["Time"] = df["Time"].astype(str)  # แปลงให้เป็น string ก่อนเพื่อตรวจสอบค่า

# แสดงค่าเริ่มต้นของคอลัมน์ Time
df_time_sample = df[["Time"]].drop_duplicates().head(20)

# แปลงคอลัมน์เวลาเป็น datetime พร้อมแสดงเวลา
df["Time"] = pd.to_datetime(df["Time"], errors='coerce')

# ละทิ้งคอลัมน์ที่ไม่จำเป็น (ข้อมูลที่ไม่เกี่ยวข้องกับ PV readings)
df_filtered_new = df.drop(columns=["Site Name", "Management Domain", "ManageObject"], errors='ignore')

# Melt ข้อมูลเพื่อให้มีคอลัมน์ Parameter และ Value
df_melted_new = df_filtered_new.melt(id_vars=["Time"], var_name="Parameter", value_name="Value")

# แยกค่า PV_no และ ประเภทของพารามิเตอร์ (กระแส/แรงดัน)
df_melted_new["PV_no."] = df_melted_new["Parameter"].str.extract(r'(\d+)').astype("Int64")
df_melted_new["Parameter"] = df_melted_new["Parameter"].str.replace(r'\d+', '', regex=True).str.strip()

# Pivot Table ให้เป็นรูปแบบที่ต้องการ
df_final_new = df_melted_new.pivot_table(index=["Time", "PV_no."], columns="Parameter", values="Value").reset_index()

# จัดเรียงคอลัมน์ให้ตรงกับตัวอย่าง
df = df_final_new.rename(columns={"input current(A)": "PV_input_current(A)", 
                                            "input voltage(V)": "PV_input_voltage(V)"})

df.head(60)


c:\Users\Acer\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Parameter,Time,PV_no.,PV input current(A),PV input voltage(V)
0,2025-02-08 00:00:00,1,0.0,0.0
1,2025-02-08 00:00:00,2,0.0,0.0
2,2025-02-08 00:00:00,3,0.0,0.0
3,2025-02-08 00:00:00,4,0.0,0.0
4,2025-02-08 00:00:00,5,0.0,0.0
5,2025-02-08 00:00:00,6,0.0,0.0
6,2025-02-08 00:00:00,7,0.0,0.0
7,2025-02-08 00:00:00,8,0.0,0.0
8,2025-02-08 00:00:00,9,0.0,0.0
9,2025-02-08 00:00:00,10,0.0,0.0


## รับข้อมูลการวัดสภาพแวดล้อมจาก EMI

In [51]:
import numpy as np
import pandas as pd
emi_uri = input("Enter data name (without extension): ")

if not emi_uri.endswith('.xlsx'):
    emi_uri += '.xlsx'

df_emi = pd.read_excel(emi_uri)
df_emi.drop(df_emi.columns[[0,1,2]],axis=1,inplace=True)
df_emi.drop([0,1,2,3],axis=0,inplace=True)
df_emi.columns = ['Time','Ambient Temp','Irradiance','PV_temp']
df_emi['Time'] = pd.to_datetime(df_emi['Time'])
df_emi.set_index(['Time'],inplace=True)
df_emi


c:\Users\Acer\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Ambient Temp,Irradiance,PV_temp
Time,,,
2025-02-08 00:15:00,0,0,24.5
2025-02-08 00:30:00,0,0,24.9
2025-02-08 00:45:00,0,0,24.5
2025-02-08 01:00:00,0,0,24.4
2025-02-08 01:15:00,0,0,25.2
...,...,...,...
2025-03-08 03:30:00,0,0,22.6
2025-03-08 03:45:00,0,0,22.9
2025-03-08 04:00:00,0,0,22.9


## รวมข้อมูลผลิตกระแสและสภาพแวดล้อม

In [52]:
merged_df = pd.merge(df,df_emi,  on='Time', how='left')
merged_df.fillna(0, inplace=True)
data = merged_df
data

C:\Users\Acer\AppData\Local\Temp\ipykernel_14424\3268465981.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df.fillna(0, inplace=True)


,Time,PV_no.,PV input current(A),PV input voltage(V),Ambient Temp,Irradiance,PV_temp
0,2025-02-08,1,0.0,0.0,0,0,0.0
1,2025-02-08,2,0.0,0.0,0,0,0.0
2,2025-02-08,3,0.0,0.0,0,0,0.0
3,2025-02-08,4,0.0,0.0,0,0,0.0
4,2025-02-08,5,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...
14115,2025-03-08,6,0.0,0.0,0,0,22.5
14116,2025-03-08,7,0.0,0.0,0,0,22.5
14117,2025-03-08,8,0.0,0.0,0,0,22.5
14118,2025-03-08,9,0.0,0.0,0,0,22.5


## บันทึก csv และ excel

In [53]:
file_uri = input("Enter data name to load (without extension): ")

data.to_csv(f"{file_uri}.csv",index_label="Time")
data.to_excel(f"{file_uri}.xlsx")

In [54]:
data

,Time,PV_no.,PV input current(A),PV input voltage(V),Ambient Temp,Irradiance,PV_temp
0,2025-02-08,1,0.0,0.0,0,0,0.0
1,2025-02-08,2,0.0,0.0,0,0,0.0
2,2025-02-08,3,0.0,0.0,0,0,0.0
3,2025-02-08,4,0.0,0.0,0,0,0.0
4,2025-02-08,5,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...
14115,2025-03-08,6,0.0,0.0,0,0,22.5
14116,2025-03-08,7,0.0,0.0,0,0,22.5
14117,2025-03-08,8,0.0,0.0,0,0,22.5
14118,2025-03-08,9,0.0,0.0,0,0,22.5
